In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/Users/danieloliveiradebrito/Projetos/fakenews


/Users/danieloliveiradebrito/Projetos/fakenews/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Load dataset

In [9]:
from fakenews.read_data import read_fakebr_corpus
import datasets
from sklearn.model_selection import train_test_split

import polars as pl

normalized_fake = read_fakebr_corpus("data/fakebr-corpus/size_normalized_texts/fake/*.txt")
normalized_true = read_fakebr_corpus("data/fakebr-corpus/size_normalized_texts/true/*.txt")

df = pl.concat([normalized_fake, normalized_true]).rename({"texts": "text"}).to_pandas()

train_full, test = train_test_split(df,
                              test_size=0.2,
                              random_state=42,
                              shuffle=True,
                              stratify=df["label"]
                            )

train, val = train_test_split(train_full,
                              test_size=0.2,
                              random_state=42,
                              shuffle=True,
                              stratify=train_full["label"]
                            )

ds = datasets.DatasetDict()
ds["train"] = datasets.Dataset.from_pandas(train.reset_index(drop=True))
ds["test"] = datasets.Dataset.from_pandas(test.reset_index(drop=True))
ds["validation"] = datasets.Dataset.from_pandas(val.reset_index(drop=True))


# BERT

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("adalbertojunior/distilbert-portuguese-cased")
tokenizer = AutoTokenizer.from_pretrained("adalbertojunior/distilbert-portuguese-cased", do_lower_case=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at adalbertojunior/distilbert-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from fakenews.train import FakeNewsTrainer

fakebr_trainer = FakeNewsTrainer(ds)
tokenized_ds = fakebr_trainer.tokenize_ds()
trainer = fakebr_trainer.get_trainer(tokenized_ds)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at adalbertojunior/distilbert-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 1152/1152 [00:00<00:00, 1322.38 examples/s]


In [11]:
trainer.train()

 29%|██▉       | 500/1728 [05:05<13:05,  1.56it/s]

{'loss': 0.283, 'grad_norm': 24.987199783325195, 'learning_rate': 1.4212962962962964e-05, 'epoch': 0.87}


                                                  
 33%|███▎      | 576/1728 [06:18<11:57,  1.61it/s]

{'eval_loss': 0.15938979387283325, 'eval_accuracy': 0.9565972222222222, 'eval_clf_report': {'0': {'precision': 0.9383333333333334, 'recall': 0.9774305555555556, 'f1-score': 0.9574829931972789, 'support': 576.0}, '1': {'precision': 0.9764492753623188, 'recall': 0.9357638888888888, 'f1-score': 0.9556737588652482, 'support': 576.0}, 'accuracy': 0.9565972222222222, 'macro avg': {'precision': 0.9573913043478262, 'recall': 0.9565972222222222, 'f1-score': 0.9565783760312636, 'support': 1152.0}, 'weighted avg': {'precision': 0.9573913043478259, 'recall': 0.9565972222222222, 'f1-score': 0.9565783760312636, 'support': 1152.0}}, 'eval_runtime': 24.7521, 'eval_samples_per_second': 46.541, 'eval_steps_per_second': 5.818, 'epoch': 1.0}


 58%|█████▊    | 1000/1728 [10:43<07:15,  1.67it/s] 

...

                                                   
100%|██████████| 1728/1728 [19:02<00:00,  1.61it/s]

{'eval_loss': 0.1877591460943222, 'eval_accuracy': 0.9652777777777778, 'eval_clf_report': {'0': {'precision': 0.9542372881355933, 'recall': 0.9774305555555556, 'f1-score': 0.9656946826758147, 'support': 576.0}, '1': {'precision': 0.9768683274021353, 'recall': 0.953125, 'f1-score': 0.9648506151142355, 'support': 576.0}, 'accuracy': 0.9652777777777778, 'macro avg': {'precision': 0.9655528077688642, 'recall': 0.9652777777777778, 'f1-score': 0.9652726488950252, 'support': 1152.0}, 'weighted avg': {'precision': 0.9655528077688642, 'recall': 0.9652777777777778, 'f1-score': 0.9652726488950252, 'support': 1152.0}}, 'eval_runtime': 24.9699, 'eval_samples_per_second': 46.136, 'eval_steps_per_second': 5.767, 'epoch': 3.0}


100%|██████████| 1728/1728 [19:04<00:00,  1.51it/s]

{'train_runtime': 1144.9333, 'train_samples_per_second': 12.074, 'train_steps_per_second': 1.509, 'train_loss': 0.152462148004108, 'epoch': 3.0}


TrainOutput(global_step=1728, training_loss=0.152462148004108, metrics={'train_runtime': 1144.9333, 'train_samples_per_second': 12.074, 'train_steps_per_second': 1.509, 'total_flos': 1831229319020544.0, 'train_loss': 0.152462148004108, 'epoch': 3.0})

In [12]:
import json

eval_dict = trainer.evaluate(tokenized_ds["test"])
with open("reports/model_eval/fakebr_eval.json", "w") as f:
    json.dump(eval_dict, f)

100%|██████████| 180/180 [00:32<00:00,  5.51it/s]
